In [ ]:

from tf_pose import common
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# Initialize pose estimator
model = 'mobilenet_thin'  # You can choose other models like 'cmu'
estimator = TfPoseEstimator(get_graph_path(model), target_size=(432, 368))

# Load an image
image_path = 'path_to_your_image.jpg'  # Replace with your image path
image = common.read_imgfile(image_path, None, None)
if image is None:
    raise Exception('Image can not be read, path=%s' % image_path)

# Estimate poses
humans = estimator.inference(image)
image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

# Show the image
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()

In [4]:
import cv2 as cv
from keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt

BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]


target_size = (1024, 1309)


net = cv.dnn.readNetFromTensorflow("graph_opt.pb")

threshold=0.2

# img = cv.imread('person_image.png',cv.IMREAD_UNCHANGED)
img = Image.open('person_image.png')
img = img.convert('RGB')
cv.imshow("initial", image.img_to_array(img))
# img = img.crop(target_size[::-1], Image.Resampling.LANCZOS)
# img = image.load_img(image_path, target_size=target_size)
img_array = image.img_to_array(img)
cropped_img_array = image.img_to_array(img.crop((0,0, 1024, 1024)))


photo_height=cropped_img_array.shape[0]
photo_width=cropped_img_array.shape[1]
net.setInput(cv.dnn.blobFromImage(cropped_img_array, 1.0, (target_size[1], target_size[0]), (127.5, 127.5, 127.5), swapRB=True, crop=False))

out = net.forward()
print(out.shape)
out = out[:, :19, :, :]
print(out.shape)
assert(len(BODY_PARTS) == out.shape[1])

points = []
for i in range(len(BODY_PARTS)):
    # Slice heatmap of corresponging body's part.
    heatMap = out[0, i, :, :]
    print(heatMap.shape)
    # Originally, we try to find all the local maximums. To simplify a sample
    # we just find a global one. However only a single pose at the same time
    # could be detected this way.
    _, conf, _, point = cv.minMaxLoc(heatMap)
    x = (photo_width * point[0]) / out.shape[3]
    y = (photo_height * point[1]) / out.shape[2]
    # Add a point if it's confidence is higher than threshold.
    points.append((int(x), int(y)) if conf > threshold else None)


for pair in POSE_PAIRS:
    partFrom = pair[0]
    partTo = pair[1]
    assert(partFrom in BODY_PARTS)
    assert(partTo in BODY_PARTS)

    idFrom = BODY_PARTS[partFrom]
    idTo = BODY_PARTS[partTo]

    if points[idFrom] and points[idTo]:
        cv.line(cropped_img_array, points[idFrom], points[idTo], (0, 255, 0), 3)
        cv.ellipse(cropped_img_array, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
        cv.ellipse(cropped_img_array, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

t, _ = net.getPerfProfile()

cv.imshow("cool",img_array)

#waits for user to press any key 
#(this is necessary to avoid Python kernel form crashing)
cv.waitKey(0)

#closing all open windows 
cv.destroyAllWindows() 

(1, 57, 128, 164)
(1, 19, 128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
(128, 164)
